In [1]:
import pandas as pd
import numpy as np

In [2]:
from collections import Counter

In [3]:
raw_data = pd.read_excel('8111B换流变A相油色谱监测数据.xlsx')
err_data = pd.read_excel('油色谱案例数据.xls')

In [4]:
def normalization(x):
    if x == '正常':
        x = 0
    else:
        x = 1
    return x

In [5]:
err_data['故障类型'] = err_data['故障类型'].apply(lambda x : normalization(x));

In [6]:
err_data['label'] = err_data.pop('故障类型')

In [7]:
Counter(err_data['label'])

Counter({0: 18, 1: 181})

In [8]:
err_data.insert(loc=5,column='total',value=err_data['CH4'] + err_data['C2H2'] + err_data['C2H4'] + err_data['C2H6'])

In [9]:
# err_data

In [10]:
CO = raw_data.pop('CO')
CO2 = raw_data.pop('CO2')
O2 = raw_data.pop('O2')
N2 = raw_data.pop('N2')
time = raw_data.pop('时间')
predicts = raw_data.pop('分类模型判别结果(0为正常点，1为异常点)')

In [11]:
raw_data['total'] = raw_data.pop('总烃')
raw_data['label'] = raw_data.pop(r'聚类模型判别结果(0为正常点，1为异常点)')

In [12]:
##------------------------------------------------------
# err_data.insert(column='H2',loc=0,value=err_data.pop('H2'))
## 新的数据集（伪）
# data = pd.concat([raw_data,err_data],sort=False)
# data = data.sample(n=data.shape[0])
# data.reset_index(drop=True).to_excel(r'监测数据异常诊断.xlsx')
##---------------------------------------------------------

In [13]:
# raw_data

In [14]:
from decimal import Decimal
def isEqual(data):
    if abs(data.CH4 + data.C2H2 +  data.C2H4 +  data.C2H6 - data.total) < 1e-1:
        return True
    else:
        return False

In [15]:
r = raw_data.apply(lambda x:isEqual(x),axis=1)

In [16]:
Counter(r)

Counter({False: 252, True: 4209})

In [17]:
error_sample = raw_data[~r].reset_index(drop=True)

In [18]:
error_sample['error'] = error_sample.CH4 + error_sample.C2H2 + error_sample.C2H4 + error_sample.C2H6 - error_sample.total 

In [19]:
abs(error_sample.error).min(),abs(error_sample.error).max()

(0.10000000000000053, 0.95100000000000051)

In [20]:
# 甲方说明：即使 总烃！=CH4 + C2H2 + C2H4 + C2H6 也不能说明设备异常
# 因此 首先按照阈值法和三比值法标注异常数据

In [21]:
# 原始监测数据中的C2H2最大值超过了阈值
raw_data.C2H2.max()

1.1519999999999999

In [22]:
# 原始监测数据中的H2最大值没有超过阈值
raw_data.H2.max()

15.199999999999999

In [23]:
# 原始监测数据中的C2H2最大值超过了阈值
raw_data.C2H2.max()

1.1519999999999999

In [24]:
warning_data = raw_data[raw_data.C2H2 - 0.8 > 1e-5]

In [25]:
# 三比值法诊断warning_data是否出现异常

def C2H2_C2H4(data):
    tmp = data.C2H2 / data.C2H4
    if tmp < 0.1:
        return 0
    elif tmp >= 3:
        return 2
    else:
        return 1

def CH4_H2(data):
    tmp = data.CH4 / data.H2
    if tmp < 0.1:
        return 1
    elif tmp >= 1:
        return 2
    else:
        return 0
    
def C2H4_C2H6(data):
    tmp = data.C2H4 /data.C2H6
    if tmp < 1:
        return 0
    elif tmp >= 3:
        return 2
    else:
        return 1

In [26]:
warning_data['C2H2/C2H4'] = warning_data.apply(lambda x : C2H2_C2H4(x),axis=1)
warning_data['CH4/H2'] = warning_data.apply(lambda x : CH4_H2(x),axis=1)
warning_data['C2H4/C2H6'] = warning_data.apply(lambda x : C2H4_C2H6(x),axis=1)

e:\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
e:\python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
e:\python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#i

In [27]:
warning_data

,H2,CH4,C2H6,C2H4,C2H2,total,label,C2H2/C2H4,CH4/H2,C2H4/C2H6
1383,15.000,8.600,1.600,1.100,1.000,12.500,0,1,0,0
1393,14.000,8.000,1.800,1.000,1.000,11.800,0,1,0,0
1403,14.000,8.800,1.100,1.100,1.000,12.500,0,1,0,1
2546,12.000,10.771,1.369,0.501,1.000,14.240,0,1,0,0
2561,11.000,9.718,1.448,0.408,1.000,12.924,0,1,0,0
4127,6.061,11.942,0.360,0.619,0.872,13.793,0,1,2,1
4230,6.061,11.722,1.822,0.333,1.152,15.028,1,2,2,0


In [28]:
err_data[err_data.H2 == 6.061]

,CH4,H2,C2H2,C2H4,C2H6,total,label


In [48]:
Counter(raw_data.label)


Counter({0: 4454, 1: 7})

In [30]:
Counter(err_data.label)

Counter({0: 18, 1: 181})

In [31]:
raw_data.label = 0

In [49]:
raw_data[raw_data.C2H2 - 0.8 > 1e-5]

,H2,CH4,C2H6,C2H4,C2H2,total,label
1383,15.000,8.600,1.600,1.100,1.000,12.500,1
1393,14.000,8.000,1.800,1.000,1.000,11.800,1
1403,14.000,8.800,1.100,1.100,1.000,12.500,1
2546,12.000,10.771,1.369,0.501,1.000,14.240,1
2561,11.000,9.718,1.448,0.408,1.000,12.924,1
4127,6.061,11.942,0.360,0.619,0.872,13.793,1
4230,6.061,11.722,1.822,0.333,1.152,15.028,1


In [47]:
raw_data.set_value(col='label',value=1,index=raw_data[raw_data.C2H2 - 0.8 > 1e-5].index)

,H2,CH4,C2H6,C2H4,C2H2,total,label
0,10.300,4.500,1.300,0.500,0.000,6.300,0
1,10.300,4.500,1.300,0.500,0.000,6.300,0
2,10.400,4.000,1.600,0.700,0.000,6.300,0
3,10.300,3.600,1.800,0.800,0.000,6.200,0
4,10.200,3.900,1.100,0.700,0.000,5.700,0
5,10.100,5.200,0.400,0.600,0.000,6.200,0
6,10.000,3.900,1.500,0.500,0.000,5.900,0
7,10.300,5.400,0.400,0.400,0.000,6.200,0
8,10.400,3.900,1.300,0.800,0.000,6.000,0
9,10.500,3.900,1.600,0.800,0.000,6.300,0


In [51]:
dataset = pd.concat([raw_data,err_data])

In [52]:
dataset = dataset.sample(n=dataset.shape[0])

In [55]:
Counter(dataset.label)

Counter({0: 4472, 1: 188})